Para actualizar un dato en una tabla de la base de datos utilizando FastAPI y MySQL, necesitarás una ruta en tu aplicación FastAPI que maneje las solicitudes de actualización y luego ejecutar una consulta SQL para realizar la actualización en la base de datos.

In [ ]:
import MySQLdb
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import uvicorn
from datetime import datetime  # Importa datetime

nest_asyncio.apply()

# Database configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'passwd': 'Rafasql24',
    'db': 'sakila',
}

# Create a connection to the database
conn = MySQLdb.connect(**db_config)

app = FastAPI()

# Pydantic model to define the schema of the data
class ItemUpdate(BaseModel):
    first_name: str
    last_name: str

# Route to update an item
@app.put("/items/{actor_id}")
def update_item(actor_id: int, item_update: ItemUpdate):
    cursor = conn.cursor()
    query = "UPDATE actor SET first_name = %s, last_name = %s WHERE actor_id = %s"
    data = (item_update.first_name, item_update.last_name, actor_id)
    cursor.execute(query, data)
    conn.commit()  # Guardar los cambios en la base de datos
    affected_rows = cursor.rowcount
    cursor.close()

    if affected_rows == 0:
        raise HTTPException(status_code=404, detail="Item not found")

    return {"message": f"Item with actor_id {actor_id} updated successfully"}

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)

En este código, hemos definido una nueva ruta PUT en la ruta /items/{actor_id} que espera un parámetro de ruta actor_id para identificar el elemento que se actualizará y los datos actualizados en forma de ItemUpdate que contiene los nuevos valores first_name y last_name.

Dentro de la función update_item, ejecutamos una consulta SQL UPDATE para actualizar los valores en la tabla actor en función del actor_id proporcionado. Después de ejecutar la consulta, confirmamos los cambios utilizando conn.commit(). Luego verificamos cuántas filas se vieron afectadas por la actualización usando cursor.rowcount. Si ninguna fila se vio afectada (lo que significa que el actor_id proporcionado no existe en la base de datos), levantamos una excepción HTTP 404. Si la actualización se realizó con éxito, devolvemos un mensaje indicando que la actualización se realizó correctamente.